# 准备config配置文件-UNet语义分割模型

同济子豪兄 2023-2-13 6-11 6-25

## 进入MMSegmentation主目录

In [1]:
import os
os.chdir('mmsegmentation')

> 也可以选择`configs`目录里的其它模型，但需要将里面的数据集配置文件修改为`../_base_/datasets/ZihaoDataset_pipeline.py`

## 载入模型config配置文件

In [2]:
from mmengine import Config
cfg = Config.fromfile('./configs/unet/unet-s5-d16_fcn_4xb4-160k_cityscapes-512x1024.py')
dataset_cfg = Config.fromfile('./configs/_base_/datasets/ZihaoDataset_pipeline.py')

## 修改config配置文件

In [3]:
# 类别个数
NUM_CLASS = 6

In [4]:
cfg.norm_cfg = dict(type='BN', requires_grad=True) # 只使用GPU时，BN取代SyncBN
cfg.crop_size = (256, 256)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.data_preprocessor.test_cfg = dict(size_divisor=128)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

# 模型 decode/auxiliary 输出头，指定为类别个数
cfg.model.decode_head.num_classes = NUM_CLASS
cfg.model.auxiliary_head.num_classes = NUM_CLASS

# Dataloader
cfg.train_dataloader = dataset_cfg.train_dataloader
cfg.val_dataloader = dataset_cfg.val_dataloader
cfg.test_dataloader = dataset_cfg.val_dataloader

# 训练 Batch Size
cfg.train_dataloader.batch_size = 4

# Evaluator
cfg.val_evaluator = dataset_cfg.val_evaluator
cfg.test_evaluator = dataset_cfg.test_evaluator

# 结果保存目录
cfg.work_dir = './work_dirs/ZihaoDataset-UNet'

cfg.train_cfg.max_iters = 40000 # 训练迭代次数

cfg.train_cfg.val_interval = 500 # 评估模型间隔

cfg.default_hooks.logger.interval = 100 # 日志记录间隔

cfg.default_hooks.checkpoint.interval = 2500 # 模型权重保存间隔

cfg.default_hooks.checkpoint.max_keep_ckpts = 1 # 最多保留几个模型权重

cfg.default_hooks.checkpoint.save_best = 'mIoU' # 保留指标最高的模型权重

# 随机数种子
cfg['randomness'] = dict(seed=0)

## 查看完整config配置文件

In [5]:
# print(cfg.pretty_text)

## 保存最终的config配置文件

In [6]:
cfg.dump('ZihaoDataset_UNet_20230626.py')